In [104]:
# FIXED INICIAL STATE

# Grid 4x4: prueba4x4.net_grid.txt (Prueba)
# Grid 20x20: navigation_1.net_grid.txt
# Grid 40x40: navigation_2.net_grid.txt
# Grid 60x60: navigation_3.net_grid.txt
# Grid 80x80: navigation_4.net_grid.txt
# Grid 100x100: navigation_5.net_grid.txt
# Grid 120x120: navigation_6.net_grid.txt

# RANDOM INICIAL STATE

# https://raw.githubusercontent.com/djsg2021utec/proyecto-1-incertidumbre/main/PoliticasFixedRandom/RandomGoalInitialState/navigation_1.net_grid.txt # Gustavo terminar


import requests
url="https://raw.githubusercontent.com/djsg2021utec/proyecto-1-incertidumbre/main/PoliticasFixedRandom/FixedGoalInitialState/navigation_1.net_grid.txt"
response = requests.get(url)
fileContent = response.text
print(fileContent)

Shape: [20, 20]

↓ ← ← ← ← ← ← ← ← ← ← ↓ ↓ ↓ ↓ ↓ ↓ ↓ □ G 
↓ □ ↑ ↑ ↑ □ ↑ ↑ ↑ ↓ □ → → → → → → → → ↑ 
↓ ← ↓ ↓ ↓ ↓ ↓ ↓ □ → → ↑ ↑ □ ↑ ↑ □ ↑ ↑ ↑ 
↓ □ → → → → → → → ↑ ↑ ↑ → → → → ↓ ↓ ← ← 
→ → ↑ ↑ ↑ □ ↑ ↑ ↑ ↑ → → ↑ □ ↑ □ → ↓ ↓ ↓ 
↑ ↑ ↑ □ ↓ ↓ ↓ ↓ ↓ □ ↓ ↓ ↓ ↓ ↓ ↓ ↓ → → ↓ 
↑ ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← □ ↓ ↓ 
↑ ↑ □ ↑ □ ↑ ↑ ↑ □ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ← ← ← 
↓ ↓ ← ← ← ← ← ← ← ← □ ↓ ← ↓ ↓ □ ↑ ↑ □ ↑ 
→ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ ↓ □ → → → → → → ↑ 
□ → → → → → → → → → → → → ↑ ↑ □ ↑ ↑ □ ↑ 
→ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ □ ↑ ↑ ↑ → → → → → ↓ 
↑ ↑ ↓ ↓ □ ↓ ↓ ↓ ↓ ↓ ↓ ↓ □ ↓ □ ↓ ↓ ↓ ↓ ↓ 
↑ ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← ← 
↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ □ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ ↑ 
↓ ↓ ↓ ↓ ↓ □ → → → → → → → → → → → → → ↑ 
→ → → → → → ↑ □ ↑ ↑ ↑ □ ↑ ↑ □ ↑ ↑ ↑ ↑ ↑ 
↑ ↑ ↑ □ ↑ ↑ ↑ ↓ ↓ ↓ → → → → → → → → → ↓ 
↑ ← ← ← ← ← ← ← ← ← □ ↓ □ ↓ ↓ ↓ □ ↓ ↓ ↓ 
▲ ↑ □ ↑ ↑ □ ↑ ↑ □ ↑ ← ← ← ← ← ← ← ← ← ← 



In [105]:
# Se quitan los elementos innecesarios del archivo y se crea una lista con strings con las mismas dimensiones y sin espacios (GridWorld_NotSpaces)
type(fileContent)
fileContentSplitted = fileContent.split("\n")
Gridworld=fileContentSplitted[2:-1]
def RemoveSpace(StringWorld):
    StringWorld=StringWorld.replace(" ","")
    return StringWorld

result = map(RemoveSpace,Gridworld)
Gridworld_NotSpaces = list(result)

for rows in Gridworld_NotSpaces:
  print(rows)

↓←←←←←←←←←←↓↓↓↓↓↓↓□G
↓□↑↑↑□↑↑↑↓□→→→→→→→→↑
↓←↓↓↓↓↓↓□→→↑↑□↑↑□↑↑↑
↓□→→→→→→→↑↑↑→→→→↓↓←←
→→↑↑↑□↑↑↑↑→→↑□↑□→↓↓↓
↑↑↑□↓↓↓↓↓□↓↓↓↓↓↓↓→→↓
↑←←←←←←←←←←←←←←←←□↓↓
↑↑□↑□↑↑↑□↑↑↑↑↑↑↑↑←←←
↓↓←←←←←←←←□↓←↓↓□↑↑□↑
→↓↓↓↓↓↓↓↓↓↓↓□→→→→→→↑
□→→→→→→→→→→→→↑↑□↑↑□↑
→↑↑↑↑↑↑↑↑↑□↑↑↑→→→→→↓
↑↑↓↓□↓↓↓↓↓↓↓□↓□↓↓↓↓↓
↑←←←←←←←←←←←←←←←←←←←
↑↑↑↑↑↑↑↑□↑↑↑↑↑↑↑↑↑↑↑
↓↓↓↓↓□→→→→→→→→→→→→→↑
→→→→→→↑□↑↑↑□↑↑□↑↑↑↑↑
↑↑↑□↑↑↑↓↓↓→→→→→→→→→↓
↑←←←←←←←←←□↓□↓↓↓□↓↓↓
▲↑□↑↑□↑↑□↑←←←←←←←←←←


In [106]:
# Se definen las acciones validas, estados iniciales, estado final y un diccionario de movimientos
acciones_validas = ['→','←','↑','↓','□']
estados_iniciales = [ '▶','◀','▲','▼'] # ▲
estado_final = 'G'
diccionario_movimientos = {'→': [0, 1], '←':[0, -1], '↑': [-1, 0], '↓':[1, 0],
                           '▶': [0, 1], '◀':[0, -1] ,'▲':[-1, 0], '▼': [1, 0]}


In [107]:
# Se importan librerías para una manejo más fácil
import numpy as np
import time

# Funciones para definir parámetros geométricos en el Grid
def horizontal_line(begin, end, y):
    return [(i, y) for i in range(begin, end, (1 if end > begin else -1))]

def vertical_line(begin, end, x):
    return [(x, i) for i in range(begin, end, (1 if end > begin else -1))]

def layer(center, level):
    center_x, center_y = center
    right_bottom = vertical_line(center_y, center_y + level, center_x + level)
    bottom_line = horizontal_line(center_x + level, center_x - level, center_y + level)
    left_line = vertical_line(center_y + level, center_y - level, center_x - level)
    top_line = horizontal_line(center_x - level, center_x + level, center_y - level)
    right_top = vertical_line(center_y-level, center_y, center_x + level)

    return right_bottom + bottom_line + left_line + top_line + right_top

def layers(center, number_levels):
    return [layer(center, level) for level in range(1, number_levels)]

def all_layers(center, number_levels):
    return [cell for single_layer in layers(center, number_levels) for cell in single_layer]

# Función para validar una coordenada
def validar_coordenada(coordenada_in):
    if coordenada_in[0] >=0 and coordenada_in[1] < len(Gridworld_NotSpaces):
        if coordenada_in[1] >= 0 and coordenada_in[0] < len(Gridworld_NotSpaces):
            if Gridworld_NotSpaces[coordenada_in[0]][coordenada_in[1]] != "□":
                return True
    
    return False

# Función para obtener coordenadas validas vecinas para un estado
def getCoordenadasValidas(coordendas_in):
    coordenadas_vecinas = []
    coord_up = [coordendas_in[0] - 1, coordendas_in[1]]
    coord_down = [coordendas_in[0] + 1, coordendas_in[1]]
    coord_right = [coordendas_in[0], coordendas_in[1] + 1]
    coord_left = [coordendas_in[0], coordendas_in[1] - 1]
    
    # arriba
    if validar_coordenada(coord_up):
        if Gridworld_NotSpaces[coord_up[0]][coord_up[1]] != "□":
            coordenadas_vecinas.append(coord_up)

    # abajo
    if validar_coordenada(coord_down):
        if Gridworld_NotSpaces[coord_down[0]][coord_down[1]] != "□":
            coordenadas_vecinas.append(coord_down)

    # derecha
    if validar_coordenada(coord_right):
        if Gridworld_NotSpaces[coord_right[0]][coord_right[1]] != "□":
            coordenadas_vecinas.append(coord_right)

    # izquierda   
    if validar_coordenada(coord_left):
        if Gridworld_NotSpaces[coord_left[0]][coord_left[1]] != "□":
            coordenadas_vecinas.append(coord_right)

    return coordenadas_vecinas

# Función para obtener el valor inicial para cada estado
def getAccionInicial(coordenadas_in):
    if Gridworld_NotSpaces[coordenadas_in[0]][coordenadas_in[1]] != "□":
        return Gridworld_NotSpaces[coordenadas_in[0]][coordenadas_in[1]]
    return Gridworld_NotSpaces[coordenadas_in[0]][coordenadas_in[1]]

# Función para obtener coordenadas validas vecinas para un estado
def getMovimientosValidos(coordendas_in):
    movimientos_validos = []
    coord_up = [coordendas_in[0] - 1, coordendas_in[1]]
    coord_down = [coordendas_in[0] + 1, coordendas_in[1]]
    coord_right = [coordendas_in[0], coordendas_in[1] + 1]
    coord_left = [coordendas_in[0], coordendas_in[1] - 1]
    
    # arriba
    if validar_coordenada(coord_up):
        if Gridworld_NotSpaces[coord_up[0]][coord_up[1]] != "□":
            movimientos_validos.append(Gridworld_NotSpaces[coord_up[0]][coord_up[1]])

    # abajo
    if validar_coordenada(coord_down):
        if Gridworld_NotSpaces[coord_down[0]][coord_down[1]] != "□":
            movimientos_validos.append(Gridworld_NotSpaces[coord_down[0]][coord_down[1]])

    # derecha
    if validar_coordenada(coord_right):
        if Gridworld_NotSpaces[coord_right[0]][coord_right[1]] != "□":
            movimientos_validos.append(Gridworld_NotSpaces[coord_right[0]][coord_right[1]])

    # izquierda   
    if validar_coordenada(coord_left):
        if Gridworld_NotSpaces[coord_left[0]][coord_left[1]] != "□":
            movimientos_validos.append(Gridworld_NotSpaces[coord_left[0]][coord_left[1]])

    return movimientos_validos

# Función para construir el diccionario para todos los estados del GridWorld
def construir_diccionario(Gridworld_NotSpaces):
  dict_estados = {}
  index_row = 0
  for i in range(len(Gridworld_NotSpaces)):
      index_column = 0
      for j in range(len(Gridworld_NotSpaces)):
          valor_nodo = float("inf")
          dict_key = str(index_row) +"-" +str(index_column)
          coordenadas = [index_row, index_column]
          if Gridworld_NotSpaces[index_row][index_column] == "G":
              #dict_key = "EstadoFinal" 
              valor_nodo = 0
          dict_estados[dict_key] = [coordenadas, getAccionInicial(coordenadas), getMovimientosValidos(coordenadas), 
                                      valor_nodo, getCoordenadasValidas(coordenadas)]
          index_column += 1
      index_row += 1
  return dict_estados

In [108]:
# Se inicializa un diccionario que guardará la información de todos los estados
# string_coordenadas: [valor_id_0, valor_id_1, valor_id_2, valor_id_3, valor_id_4]
# clave: string_coordenadas
# valor: lista de valores que guardan la información del estado
#   valor_id_0: list_coordenadas
#   valor_id_1: chr_accion
#   valor_id_2: list_acciones_permitidas
#   valor_id_3: float_valor
#   valor_id_4: lista_acciones_permitidas_en_coordenadas


# Se construye el diccionario
dict_estados = construir_diccionario(Gridworld_NotSpaces)

In [109]:
## Se definen los parámetros
costo_accion = 1 # Costo de cada accion
valor_final = 0 # Valor del estado final
limite_superior = str(len(Gridworld_NotSpaces)-1) #Límite superior para el diccionario de estados
coordenadas_finales = dict_estados["0-"+limite_superior][0] #Las coordendas finales

In [110]:

inicio = time.time()

all_coordinates = all_layers(coordenadas_finales, len(Gridworld_NotSpaces))
numero_iteraciones = 0

is_politica_estable = False

while not(is_politica_estable):
  politica_actual = [valor[3] for valor in dict_estados.values()]

  # Entramos a las coordenadas resultantes de aplicar la política actual a los estados
  for coord in all_coordinates:
      #--------------------------------
      #| 1. Evaluación de la Política |
      #--------------------------------
      # Se convierten las coordenadas de tuplas a listas para su editarlas fácilmente
      coord = list(coord)

      # Se valida que la coordenada no sea un pared ni salga de los límites del grid
      if validar_coordenada(coord):
        
          # Se genera la clave del estado a partir de sus coordenadas para extraer su información 
          key_coord = str(coord[0]) +'-'+str(coord[1])

          # Se obtiene la coordenada del estado siguiente a partir de las coordenadas del estado y su política actual
          coord_siguiente_estado = np.array(coord) + np.array(diccionario_movimientos[Gridworld_NotSpaces[coord[0]][coord[1]]])

          # Se genera la clave del estado siguiente a partir de sus coordenadas para extraer su información 
          key_coord_siguiente = str(coord_siguiente_estado[0])+'-'+str(coord_siguiente_estado[1]) 

          # Se obtiene el valor del estado siguiente a fin de obtener el valor del estado actual
          valor_coord_siguiente = dict_estados[key_coord_siguiente][3]

          # Se calcula el valor actual del estado según la política
          valor_actual_estado = costo_accion + valor_coord_siguiente

          # El valor actual se actualiza en el diccionario
          dict_estados[key_coord][3] = valor_actual_estado

          #----------------------------
          #| 2. Mejora de la Política |
          #----------------------------
          # Se inicializan los políticas posibles del estado en un diccionario temporal
          dict_mov_temp=  {('→',(0, 1)): float('inf'), 
                          ('←',(0, -1)): float('inf'), 
                          ('↑',(-1,0)): float('inf'), 
                          ('↓',(1,0)): float('inf')}
          

          # Se inicializa una lista de valores de estados posibles según un cambio de política 
          list_valores_posibles = [valor_actual_estado]

          # Se iteran con todas los políticas posibles
          for otra_politica_key in dict_mov_temp.keys():

              # Se obtienen las coordenadas de otro estado a partir de la coordenada actual y la otra política posible
              coord_otro_estado = np.array(coord) + np.array(otra_politica_key[1])

              # Se valida que la coordenada no sea un pared ni salga de los límites del grid
              if validar_coordenada(list(coord_otro_estado)):
                  
                  # Se genera la clave del otro estado a partir de sus coordenadas
                  key_otro_estado = str(coord_otro_estado[0]) + '-'+str(coord_otro_estado[1])
                  
                  # Se obtiene el valor del estado posible
                  valor_otro_estado = dict_estados[key_otro_estado][3]
                  
                  # Se calcula el valor del estado actual en base al otro estado
                  valor_actual_otro_estado = costo_accion + valor_otro_estado
                  
                  # Se guarda el valor del estado actual en base a otro estado en la lista de valores posibles
                  list_valores_posibles.append(valor_actual_otro_estado)

                  # Se actualiza el diccionario temporal con el valor actual de la política en base a otro estado
                  dict_mov_temp[otra_politica_key] = valor_actual_otro_estado

                  # Se obtiene el menor valor de las posibles políticas
                  valor_politica_optima = min(list_valores_posibles)

          # Con el menor valor, se extrae la mejor politica
          politica_optima = list(dict_mov_temp.keys())[list(dict_mov_temp.values()).index(valor_politica_optima)][0]

          # Se actualiza el diccionario de los estados con la política óptima y valor de la política óptima 
          dict_estados[key_coord][1] = politica_optima
          dict_estados[key_coord][3] = valor_politica_optima


  politica_nueva = [valor[3] for valor in dict_estados.values()]

  if politica_actual == politica_nueva:
    is_politica_estable = True
  else:
    numero_iteraciones += 1

fin = time.time()
tiempo_transcurrido = fin - inicio

print("Tiempo transcurrido:", tiempo_transcurrido, "segundos", "| Número de interaciones: ",numero_iteraciones)

Tiempo transcurrido: 0.08129715919494629 segundos | Número de interaciones:  4


In [111]:
for rows in Gridworld_NotSpaces:
  print(rows)

↓←←←←←←←←←←↓↓↓↓↓↓↓□G
↓□↑↑↑□↑↑↑↓□→→→→→→→→↑
↓←↓↓↓↓↓↓□→→↑↑□↑↑□↑↑↑
↓□→→→→→→→↑↑↑→→→→↓↓←←
→→↑↑↑□↑↑↑↑→→↑□↑□→↓↓↓
↑↑↑□↓↓↓↓↓□↓↓↓↓↓↓↓→→↓
↑←←←←←←←←←←←←←←←←□↓↓
↑↑□↑□↑↑↑□↑↑↑↑↑↑↑↑←←←
↓↓←←←←←←←←□↓←↓↓□↑↑□↑
→↓↓↓↓↓↓↓↓↓↓↓□→→→→→→↑
□→→→→→→→→→→→→↑↑□↑↑□↑
→↑↑↑↑↑↑↑↑↑□↑↑↑→→→→→↓
↑↑↓↓□↓↓↓↓↓↓↓□↓□↓↓↓↓↓
↑←←←←←←←←←←←←←←←←←←←
↑↑↑↑↑↑↑↑□↑↑↑↑↑↑↑↑↑↑↑
↓↓↓↓↓□→→→→→→→→→→→→→↑
→→→→→→↑□↑↑↑□↑↑□↑↑↑↑↑
↑↑↑□↑↑↑↓↓↓→→→→→→→→→↓
↑←←←←←←←←←□↓□↓↓↓□↓↓↓
▲↑□↑↑□↑↑□↑←←←←←←←←←←


In [112]:
Gridworld_Optimizado = np.array([value[1] for value in dict_estados.values()]).reshape(len(Gridworld_NotSpaces),len(Gridworld_NotSpaces))
for row in Gridworld_Optimizado:
    print(''.join(row))

→→→→→→→→→→→→→→→→→↓□G
↑□→→↑□→→→↑□→→→→→→→→↑
→→→→→→→↑□→→→↑□→↑□→→↑
↑□→→→→→→→→→→→→→→→→→↑
→→→→↑□→→→→→→↑□↑□→→→↑
→→↑□→→→→↑□→→→→→→→→→↑
→→→→→→→→→→→→→→→→↑□→↑
→↑□↑□→→↑□→→→→→→→→→→↑
→→→→→→→→→↑□→→→↑□→↑□↑
→→→→→→→→→→→↑□→→→→→→↑
□→→→→→→→→→→→→→↑□→↑□↑
→→→→→→→→→↑□→→→→→→→→↑
→→→↑□→→→→→→↑□↑□→→→→↑
→→→→→→→→→→→→→→→→→→→↑
→→→→→→→↑□→→→→→→→→→→↑
→→→→↑□→→→→→→→→→→→→→↑
→→→→→→↑□→→↑□→↑□→→→→↑
→→↑□→→→→→→→→→→→→→→→↑
→→→→→→→→→↑□↑□→→↑□→→↑
→↑□→↑□→↑□→→→→→→→→→→↑
